# Semantic representation of user utterance

We'll process the example dataset utterance:

```
show me flights from denver to philadelphia on tuesday
```

In [63]:
import spacy
from spacy.matcher import Matcher
import json

nlp = spacy.load('en_core_web_lg')
utterance = 'show me flights from denver to philadelphia on tuesday'
doc = nlp(utterance)

def extract_location_entities(doc):
    matcher = Matcher(nlp.vocab)
    pattern = [{'POS': 'ADP'}, {'ENT_TYPE': 'GPE'}]
    matcher.add('Location', [pattern])
    matches = matcher(doc)
    spans = [doc[start:end] for mid, start, end in matches]
    locations = {l[0].text: l[1].text for l in spans}
    return locations

def extract_date_entities(doc):
    matcher = Matcher(nlp.vocab)
    pattern = [{'ENT_TYPE': 'DATE'}]
    matcher.add('Date', [pattern])
    matches = matcher(doc)
    spans = [doc[start:end] for mid, start, end in matches]
    return spans[0].text if len(spans) > 0 else ''

def detect_intent(doc):
    for token in doc:
        if token.dep_ == "dobj":
            return token.head.lemma_ + token.lemma_.capitalize()
    return ''

locations = extract_location_entities(doc)
date = extract_date_entities(doc)
entities = {'locations': locations, 'date': date}
intent = detect_intent(doc)
representation = {'utterance': utterance, 'intent': intent, 'entities': entities}

print(json.dumps(representation, indent=4))

{
    "utterance": "show me flights from denver to philadelphia on tuesday",
    "intent": "showFlight",
    "entities": {
        "locations": {
            "from": "denver",
            "to": "philadelphia"
        },
        "date": "tuesday"
    }
}


We got a complete semantic representation out of an utterance. This is a machine-readable and usable output. NLP ends here. At this point we can create a practical application which gets as input this json and generates an appropriate response action.